In Python, functions are first-class obejcts. This means that functions can be used as arguments just like any other objects.<br>
Decorators or Wrap: accepts functions and returns functions. <br>

In [5]:
#Inner functions: function can be defined inseide the functions;

def parent():
    print("parent")
    
    def first_child():
        print("first child ")
    def second_child():
        print("second child")
        
    second_child()
    first_child()
    
print( parent() ) # works

print( second_child() ) # Errors

parent
second child
first child 
None


NameError: name 'second_child' is not defined

In [9]:
def parent(num):
    def first_child():
        return "HI, its me"
    def second_child():
        return "Hi, call me"
    
    if num == 1:
        return first_child
    else:
        return second_child
    
first = parent(1)
first()

'HI, its me'

In [18]:
def check_positive(func):
   def func_wrapper(x, y):
      if x<0 or y<0:
         raise Exception("Both x and y have to be positive for function {} to work".format(func.__name__))
      res = func(x,y)
      return res
   return func_wrapper

@check_positive
def average(x, y):
   return (x + y)/2

In [20]:
average(-1,0)

Exception: Both x and y have to be positive for function average to work

In [30]:
def validateDictionary(original_function):
    def new_function(dictionary, *args, **kwargs):
        if isinstance(dictionary, dict) == True:
            print('Is a dictionary')
            return original_function(dictionary, *args, **kwargs)
        else:
            print("Not a dictionary")
            return []
        return new_function
 
class validLengthDictionary(object):
    def __init__(self,length):
        self.length = length
    def __call__(self, original_function):
        def new_function( dictionary, *args, **kwargs):
            if len(dictionary) >= self.length:
                print( len(dictionary),  "Length")
                return original_function(dictionary, *args, **kwargs)
            else:
                print("Not long enough")
                return original_function({}, *args, **kwargs)
        return new_function
    
@validateDictionary
@validLengthDictionary(2)
def getKeys(dictionary, *args, **kwargs):
    return list( dictionary.keys() )

In [72]:
#Synchronization the Locks
import functools

def synchronized(lock):
    """ Synchronization decorator """
    def wrap(f):
        @functools.wraps(f)
        def newFunction(*args, **kw):
            with lock:
                return f(*args, **kw)
        return newFunction
    return wrap


import threading
lock = threading.Lock()

@synchronized(lock)
def test():
    print("hello")
    


In [73]:
#validate the IMEI number
def validate_summary(func):
    def wrapper(*args, **kwargs):
        data = func(*args, **kwargs)
        if  data["summary"] > 80:
            raise ValueError("Summary too long")
        return data
    return wrapper

@validate_summary
def fetch_customer_data():
    data = {"summary" : 81 }
    return data

fetch_customer_data()

ValueError: Summary too long

In [74]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer


@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])
        
waste_some_time(15)

Finished 'waste_some_time' in 0.0988 secs


In [75]:
#slow down the function

import functools
import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)
        
countdown(10)

10
9
8
7
6
5
4
3
2
1
Liftoff!


In [76]:
#Statefull decorators
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Whee!")
    
say_whee()
say_whee()

Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!


In [77]:
import functools
import time

def slow_down(_func=None, *, rate=1):
    """Sleep given amount of seconds before calling the function"""
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)
    
@slow_down(rate=2)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)
        
countdown(3)

3
2
1
Liftoff!


In [78]:
#Singletons: A singleton is a class with only one instance. There are several singletons in Python that you use frequently, including None, True, and False. It is the fact that None is a singleton that allows you to compare for None using the is keyword

import functools

def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

@singleton
class TheOne:
    pass



In [79]:
first_one = TheOne()
print ( id(first_one) )

another_one = TheOne()
print( id(another_one))

2276548567504
2276548567504
